# Imports

In [1]:
from regression_model_comparison import RegressionModelComparison
import numpy as np
import pandas as pd
import mlflow
import time

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Data

In [3]:
dataX = pd.read_csv('data/engie_X.csv', header=0, sep=';', decimal='.')
# dataX.info()

In [4]:
dataY = pd.read_csv('data/engie_Y.csv', header=0,  sep=';', decimal='.')
dataY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617386 entries, 0 to 617385
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      617386 non-null  int64  
 1   TARGET  617386 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 9.4 MB


In [5]:
data_raw = pd.merge(dataX, dataY, on='ID', how='inner')
print("SHAPE = ", data_raw.shape)
data_raw.head(3)

SHAPE =  (617386, 79)


,ID,MAC_CODE,Date_time,Pitch_angle,Pitch_angle_min,Pitch_angle_max,Pitch_angle_std,Hub_temperature,Hub_temperature_min,Hub_temperature_max,...,Rotor_speed_min,Rotor_speed_max,Rotor_speed_std,Rotor_bearing_temperature,Rotor_bearing_temperature_min,Rotor_bearing_temperature_max,Rotor_bearing_temperature_std,Absolute_wind_direction_c,Nacelle_angle_c,TARGET
0,1,WT3,1.0,92.470001,92.470001,92.470001,0.0,7.0,7.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,294.19000,294.23999,-0.703
1,2,WT3,2.0,92.470001,92.470001,92.470001,0.0,7.0,7.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,297.82999,294.23999,-0.747
2,3,WT3,3.0,92.470001,92.470001,92.470001,0.0,7.0,7.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,322.20999,294.23999,-0.791


## Séparation des éoliennes

In [6]:
data_raw.groupby('MAC_CODE').agg(nrows=('MAC_CODE', 'count'))

,nrows
MAC_CODE,
WT1,154707
WT2,154791
WT3,154253
WT4,153635


## Observation des données

In [7]:
# if np.isnan(data_raw).any():
#     raise ValueError("Il y a des valeurs manquantes (NaN) dans les données")
# else:
#     print("Aucune valeur manquante (NaN) dans les données")

In [8]:
# import statsmodels.api as sm

# #Fit linear model to any dataset
# model = sm.OLS(Y, X)
# results = model.fit()

# #create instance of influence
# influence = results.get_influence()

# #leverage (hat values)
# leverage = influence.hat_matrix_diag

## Définition des features numériques et des features tansformables

In [9]:
numerical_features = [
    'Date_time', 'Pitch_angle_std', 'Hub_temperature',
       'Hub_temperature_min', 'Hub_temperature_max', 'Hub_temperature_std',
       'Generator_converter_speed', 'Generator_converter_speed_min',
       'Generator_converter_speed_max', 'Generator_converter_speed_std',
       'Generator_speed', 'Generator_speed_min', 'Generator_speed_max',
       'Generator_speed_std', 'Generator_bearing_1_temperature',
       'Generator_bearing_1_temperature_min',
       'Generator_bearing_1_temperature_max',
       'Generator_bearing_1_temperature_std',
       'Generator_bearing_2_temperature',
       'Generator_bearing_2_temperature_min',
       'Generator_bearing_2_temperature_max',
       'Generator_bearing_2_temperature_std', 'Generator_stator_temperature',
       'Generator_stator_temperature_min', 'Generator_stator_temperature_max',
       'Generator_stator_temperature_std', 'Gearbox_bearing_1_temperature',
       'Gearbox_bearing_1_temperature_min',
       'Gearbox_bearing_1_temperature_max',
       'Gearbox_bearing_1_temperature_std', 'Gearbox_bearing_2_temperature',
       'Gearbox_bearing_2_temperature_min',
       'Gearbox_bearing_2_temperature_max',
       'Gearbox_bearing_2_temperature_std', 'Gearbox_inlet_temperature',
       'Gearbox_inlet_temperature_min', 'Gearbox_inlet_temperature_max',
       'Gearbox_inlet_temperature_std', 'Gearbox_oil_sump_temperature',
       'Gearbox_oil_sump_temperature_min', 'Gearbox_oil_sump_temperature_max',
       'Gearbox_oil_sump_temperature_std', 'Nacelle_angle',
       'Nacelle_angle_min', 'Nacelle_angle_max', 'Nacelle_angle_std',
       'Nacelle_temperature', 'Nacelle_temperature_min',
       'Nacelle_temperature_max', 'Nacelle_temperature_std',
       'Absolute_wind_direction', 'Outdoor_temperature',
       'Outdoor_temperature_min', 'Outdoor_temperature_max',
       'Outdoor_temperature_std', 'Grid_frequency', 'Grid_frequency_min',
       'Grid_frequency_max', 'Grid_frequency_std', 'Grid_voltage',
       'Grid_voltage_min', 'Grid_voltage_max', 'Grid_voltage_std',
       'Rotor_speed', 'Rotor_speed_min', 'Rotor_speed_max', 'Rotor_speed_std',
       'Rotor_bearing_temperature', 'Rotor_bearing_temperature_min',
       'Rotor_bearing_temperature_max', 'Rotor_bearing_temperature_std'
       ]

other_features = ['Pitch_angle', 'Pitch_angle_min', 'Pitch_angle_max', 'Absolute_wind_direction_c', 'Nacelle_angle_c']

# Modelisation

Objectif = Un modèle gagnant pour chaque éolienne  

Il serait judicieux de séparer les JDD de chaque éolienne en groupes via clustering (car les JDD sont encore gros : 150k) pour trouver des groupes au sein des JDD de chaque éolienne.  

En attendant un modèle gagnant par éolienne, puis on prédit sur toutes les lignes de chaque éolienne, à la fin on a nos 350k prévisions et on calcule la MAE finale

## Echantillon d'essais et de debuguage du module de comparaison des modèles

In [10]:
sample_size = 50
X = data_raw.drop(columns=['ID', 'MAC_CODE', 'TARGET']).iloc[:sample_size, :]
Y = data_raw.TARGET.iloc[:sample_size]

In [11]:
comparison = RegressionModelComparison(
    X,
    Y,
    scorings=['mae', 'mse'],
    test_size=0.1,
    seed=3,
    mlflow=False
    )

##### Splitting Dataset with test_size = 0.1 and random_state = 3


In [12]:
comparison.preprocessing(
    numerical_features,
    other_features,
    nknots=4,
    poly_order=2,
    scaler='standard'
    )

In [13]:
df_results = comparison.run_comparison(
                    preproc=['base'],
                    model_param={
                        'linear_regression': {},
                        # 'ridge': {},
                        'lasso': {},
                        'elasticnet': {},
                        'randomforest': {
                            'regressor__n_estimators' : [100, 1000], # Nombre d'arbres dans la forêt. defaut 100
                            'regressor__max_depth' : [None, 30], # Profondeur maximale des arbres. Si None, les arbres sont développés jusqu'à ce que toutes les feuilles soient pures ou que chaque feuille contienne moins que min_samples_split échantillons
                            'regressor__max_features': [3, 'sqrt'], # Nombre maximum de caractéristiques considérées pour chaque split (division d'un nœud en deux sous-nœuds)
                            },
                        # 'grd_boosting': {
                        #     'regressor__learning_rate' : [.01, 1],
                        #     'regressor__max_depth' : [3, 9],
                        #     'regressor__subsample' : [0.5, 1],
                        #     'regressor__n_estimators' : [100, 1000]
                        #     }
                        },
                    nfolds=5,
                    verbose=False
                    )

###### Start comparison ######
Using preprocessor : base
Using regressor : linear_regression


Prevision score using ##mae## on test set = 1.4500682698514922
Prevision score using ##mse## on test set = 5.89904821731908
Using regressor : lasso


/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.060e-02, tolerance: 7.373e-03
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.061e-02, tolerance: 7.373e-03
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.032e-02, tolerance: 7.373e

Prevision score using ##mae## on test set = 0.7618128521872551
Prevision score using ##mse## on test set = 1.7537967567296486
Using regressor : elasticnet


/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.765e-03, tolerance: 5.309e-03
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.765e-03, tolerance: 5.309e-03
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.649e-03, tolerance: 5.309e

Prevision score using ##mae## on test set = 0.7600878864196468
Prevision score using ##mse## on test set = 1.802528795053948
Using regressor : randomforest
Prevision score using ##mae## on test set = 0.5569960444839992
Prevision score using ##mse## on test set = 0.9697552507996349
Total duration of comparison = 0.3082677443822225 minutes


In [14]:
df_results

,model_name,mae_test,mae_train,mse_test,mse_train,model,params
0,linear_regression,1.450068,3.157351e-15,5.899048,1.428302e-29,"(ColumnTransformer(transformers=[('num',\n ...",{}
1,lasso,0.761813,3.696476e-01,1.753797,4.940812e-01,"(ColumnTransformer(transformers=[('num',\n ...","{'memory': None, 'steps': [('preproc', ColumnT..."
2,elasticnet,0.760088,3.789331e-01,1.802529,5.346055e-01,"(ColumnTransformer(transformers=[('num',\n ...","{'memory': None, 'steps': [('preproc', ColumnT..."
3,randomforest,0.556996,1.757727e-01,0.969755,1.342049e-01,"(ColumnTransformer(transformers=[('num',\n ...","{'regressor__max_depth': None, 'regressor__max..."


## Comparaison complète

In [ ]:
start_time = time.time()

sample_size = 5000 # Taille du sample de dataset qui permet le choix des hyperparamètres
seed = 3
test_size = 0.1 # Taille du jeu de test pour la recherche sur grille des meilleurs modèles et hyperparamètres
metrics = ['mae', 'mse']

results_wt = {} # Dictionnaire de résultats pour l'ensemble des éoliennes
y_pred_wt = {} # Y prédit pour l'ensemble des éoliennes

array_wt = data_raw.MAC_CODE.unique() # Valeurs uniques des noms d'éoliennes
array_wt.sort()

for eolienne in array_wt: # Pour chaque éolienne
    print(f"### Comparison for wind turbine {eolienne}")

    results_wt[eolienne] = {} # Résultats propres à l'éolienne

    data_wt = data_raw[data_raw.MAC_CODE == eolienne] # Données propres à l'éolienne
    print(f"Shape of complete dataframe for wind turbine {eolienne} = {data_wt.shape}")

    # Extrait d'un échantillon de taille sample_size pour la comparaison de modèles et recherche sur grille
    df_sample = data_wt.drop(columns=['ID', 'MAC_CODE']).sample(sample_size, random_state=seed)
    X = df_sample.drop(columns=['TARGET'])
    Y = df_sample.TARGET

    # 1ère comparaison de modèles sur le sample
    comparison = RegressionModelComparison(
        X,
        Y,
        scorings=metrics,
        test_size=test_size,
        seed=seed,
        mlflow=False
        )

    # Préparation des preprocesseurs
    comparison.preprocessing(
        numerical_features,
        other_features,
        nknots=4, # Nombre de knot des features splines
        poly_order=2, # Degré des polynomes des features polynomiales
        scaler='standard', # Scaler des features numériques
        verbose=False
        )
    
    df_results = comparison.run_comparison(
        preproc=['base', 'poly'], # Préprocesseurs à tester (base, poly ou splines)
        model_param={
            'linear_regression': {},
            # 'ridge': {}, # Mon implémentation de la regression Ridge ne fonctionne pas pour le moment
            'lasso': {},
            'elasticnet': {},
            'randomforest': {
                'regressor__n_estimators' : [100, 300], # Nombre d'arbres dans la forêt. defaut 100
                'regressor__max_depth' : [None, 20], # Profondeur maximale des arbres. Si None, les arbres sont développés jusqu'à ce que toutes les feuilles soient pures ou que chaque feuille contienne moins que min_samples_split échantillons
                'regressor__max_features': [3, 'sqrt'], # Nombre maximum de caractéristiques considérées pour chaque split (division d'un nœud en deux sous-nœuds)
                },
            'grd_boosting': {
                'regressor__learning_rate' : [.01, 1],
                'regressor__max_depth' : [3, 7],
                'regressor__subsample' : [0.5, 1],
                'regressor__n_estimators' : [100, 300]
                }
            },
        nfolds=5,
        verbose=False
        )
    
    # Selection du meilleur modèle selon la MAE sur le jeu de test du dataset sample lors de la recherche sur grille
    best = df_results[df_results.mae_test == df_results.mae_test.min()]
    best_model = best.model.iloc[0]

    # Résultats intermédiaires
    results_wt[eolienne]['best_model'] = best_model # Extrait du meilleur modèle sur sample pour l'éolienne
    results_wt[eolienne]['best_params'] = best.params.iloc[0]

    # Préparation de l'ensemble du dataset de l'éolienne
    X_full = data_wt.drop(columns=['ID', 'MAC_CODE', 'TARGET'])
    Y_full = data_wt.loc[:, ['ID', 'TARGET']] # On garde l'ID pour reconstruire le vecteur Y final

    # Train/test split sur l'ensemble du dataset de l'éolienne avec test_size = 0.3
    X_train_wt, X_test_wt, y_train_wt, y_test_wt = train_test_split(X_full, Y_full.TARGET, test_size=0.3, random_state=seed)

    # Entraînement et test sur l'ensemble du dataset
    best_model.fit(X_train_wt, y_train_wt)
    estimation = best_model.predict(X_train_wt)
    prevision = best_model.predict(X_test_wt)

    # Sauvegarde du y_pred de l'éolienne pour scoring final
    y_pred = best_model.predict(X_full)
    df_pred = Y_full[['ID']].copy()
    df_pred['y_pred'] = y_pred # Concaténation avec l'ID du dataset complet pour reconstruction
    y_pred_wt[eolienne] = df_pred

    scores = {}
    for metric in metrics: 
        scores[metric] = {}
        if metric == 'mae':
            scores[metric]['train'] = mean_absolute_error(y_true=y_train_wt, y_pred=estimation)
            scores[metric]['test'] = mean_absolute_error(y_true=y_test_wt, y_pred=prevision)            
        elif metric == 'mse':
            scores[metric]['train'] = mean_squared_error(y_true=y_train_wt, y_pred=estimation)
            scores[metric]['test'] = mean_squared_error(y_true=y_test_wt, y_pred=prevision)
    
    results_wt[eolienne]['metrics'] = scores

duration = time.time() - start_time
print(f"Total duration of computation = {duration / 60} minutes")

### Comparison for wind turbine WT1
Shape of complete dataframe for wind turbine WT1 = (154707, 79)
##### Splitting Dataset with test_size = 0.1 and random_state = 3
###### Start comparison ######
Using preprocessor : base
Using regressor : linear_regression
Prevision score using ##mae## on test set = 78.06196971116955
Prevision score using ##mse## on test set = 11958.7694837509
Using regressor : lasso


/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 85262.7583836168, tolerance: 78039.92570716445
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 128269.46310085803, tolerance: 78039.92570716445
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 180194.19129806757, tolerance: 78039.92570716445
  model = cd_fast.enet_coordinate_descent_gram(
/opt/conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWa

Prevision score using ##mae## on test set = 92.93546647290928
Prevision score using ##mse## on test set = 17859.197626134865
Using regressor : elasticnet
Prevision score using ##mae## on test set = 120.56734929581225
Prevision score using ##mse## on test set = 27664.609363645144
Using regressor : randomforest


In [ ]:
data_results = []
index_results = ['wt', 'model', 'mae_train', 'mae_test', 'mse_train', 'mse_test', 'params']

for wt in results_wt.keys():
    data_results_wt = [wt]
    best_model = str(type(results_wt[wt]['best_model'].named_steps['regressor'])).split('.')[-1]
    data_results_wt.append(best_model)

    for metric in results_wt[wt]['metrics'].keys():
        for key, value in results_wt[wt]['metrics'][metric].items():
            data_results_wt.append(value)

    data_results_wt.append(results_wt[wt]['best_params'])

    data_results.append(data_results_wt)

df_results_wt = pd.DataFrame(data_results, columns=index_results)
df_results_wt

,wt,model,mae_train,mae_test,mse_train,mse_test,params
0,WT1,GradientBoostingRegressor'>,0.114123,0.571768,0.017147,0.425393,"{'regressor__learning_rate': 0.01, 'regressor_..."
1,WT2,GradientBoostingRegressor'>,0.265886,0.794905,0.122555,1.085044,"{'regressor__learning_rate': 0.01, 'regressor_..."
2,WT3,RandomForestRegressor'>,0.220082,0.432652,0.242846,0.468738,"{'regressor__max_depth': 20, 'regressor__max_f..."
3,WT4,GradientBoostingRegressor'>,0.046594,0.290987,0.003317,0.206387,"{'regressor__learning_rate': 0.01, 'regressor_..."


Construction du Y final

In [ ]:
y_pred_final = pd.concat([y_pred_wt['WT1'], y_pred_wt['WT2'], y_pred_wt['WT3'], y_pred_wt['WT4']], axis=0)
y_pred_final.sort_values(by=['ID'], inplace=True)
print(y_pred_final.shape)

(200, 2)


In [ ]:
mae_final = mean_absolute_error(y_true=data_raw.TARGET, y_pred=y_pred_final.y_pred)
mse_final = mean_squared_error(y_true=data_raw.TARGET, y_pred=y_pred_final.y_pred)

print(f"MAE finale sur l'ensemble du dataset : {mae_final}")
print(f"MSE finale sur l'ensemble du dataset : {mse_final}")

MAE finale sur l'ensemble du dataset : 1.7525330913587063
MSE finale sur l'ensemble du dataset : 5.687624175402289
